In [1]:
import mlrun


In [2]:
mlrun.set_environment("http://localhost:8080", artifact_path="./")
project = mlrun.get_or_create_project("start", "./", user_project=True)
data_gen_fn = project.set_function(
"data_prep.py",
name="data_prep",
kind="job",
image="mlrun/mlrun",
handler="breast_cancer_generator",
)
project.save()  # save the project with the latest config

gen_data_run = project.run_function("data_prep", local=True)

> 2023-05-23 20:14:56,948 [info] loaded project start from MLRun DB
> 2023-05-23 20:14:56,987 [info] starting run data-prep-breast_cancer_generator uid=f769ece7c98b4d598ed97998ae117f1f DB=http://localhost:8080


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
start-codespace,...117f1f,0,May 23 20:14:57,completed,data-prep-breast_cancer_generator,kind=owner=codespacehost=codespaces-4369a3,,,label_column=label,dataset


> 2023-05-23 20:15:02,990 [info] run executed, status=completed


In [3]:
trainer = mlrun.import_function("hub://auto_trainer")

trainer_run = project.run_function(
    trainer,
    inputs={"dataset": gen_data_run.outputs["dataset"]},
    params={
        "model_class": "sklearn.ensemble.RandomForestClassifier",
        "train_test_split_size": 0.2,
        "label_columns": "label",
        "model_name": "cancer",
    },
    handler="train",
    )

> 2023-05-23 20:15:11,859 [info] starting run auto-trainer-train uid=4d455e75079346f0a45fb2ffd299eb2f DB=http://localhost:8080
> 2023-05-23 20:15:13,413 [info] Sample set not given, using the whole training set as the sample set
> 2023-05-23 20:15:15,869 [info] training 'cancer'


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
start-codespace,...99eb2f,0,May 23 20:15:12,completed,auto-trainer-train,kind=owner=codespacehost=codespaces-4369a3,dataset,model_class=sklearn.ensemble.RandomForestClassifiertrain_test_split_size=0.2label_columns=labelmodel_name=cancer,accuracy=0.956140350877193f1_score=0.965034965034965precision_score=0.9583333333333334recall_score=0.971830985915493,feature-importancetest_setconfusion-matrixroc-curvescalibration-curvemodel


> 2023-05-23 20:15:18,774 [info] run executed, status=completed


In [7]:
trainer_run.outputs

{'accuracy': 0.956140350877193,
 'f1_score': 0.965034965034965,
 'precision_score': 0.9583333333333334,
 'recall_score': 0.971830985915493,
 'feature-importance': '/workspaces/MLRun-example/01_start/auto-trainer-train/0/feature-importance.html',
 'test_set': 'store://artifacts/start-codespace/auto-trainer-train_test_set:4d455e75079346f0a45fb2ffd299eb2f',
 'confusion-matrix': '/workspaces/MLRun-example/01_start/auto-trainer-train/0/confusion-matrix.html',
 'roc-curves': '/workspaces/MLRun-example/01_start/auto-trainer-train/0/roc-curves.html',
 'calibration-curve': '/workspaces/MLRun-example/01_start/auto-trainer-train/0/calibration-curve.html',
 'model': 'store://artifacts/start-codespace/cancer:4d455e75079346f0a45fb2ffd299eb2f'}